In [15]:
import os
import re
import json
from tqdm import tqdm

In [9]:
# dictionary
dictionary = json.load(open("words_dictionary.json", "r"))
print("alpha" in dictionary)

True


In [82]:
folder = "The Black Tulip, by Alexandre Dumas"
file = "the_black_tulip.txt"

end_pattern = r"End of Project Gutenberg's The Black Tulip"
chapter_pattern = r"^Chapter \d+."

book = {}

# create book
chapter = ""
content = ""
with open(os.path.join(folder, file), "r") as file:
    for line in file:
        line = line.strip()
        if re.search(chapter_pattern, line) is not None:
            # end of chapter
            if chapter != "":
                # refine content
                content = content.strip().lower()
                book[chapter] = content
            # start new chapter
            chapter = line
            content = ""
        elif re.search(end_pattern, line) is not None:
            # end of chapter
            if chapter != "":
                # refine content
                content = content.strip().lower()
                book[chapter] = content
            # end of book
            break
        else:
            # add new chapter content
            content += f" {line}"

In [83]:
ignore_key = ["'"]
special_key = []
# analyze book
for chapter, content in tqdm(book.items()):
    words = content.split(" ")
    for word in words:
        special_word = re.sub(r"[\w]", "", word)
        for special_char in special_word:
            if special_char not in special_key and special_char not in ignore_key:
                special_key.append(special_char)
"".join(special_key)

100%|██████████| 33/33 [00:00<00:00, 299.76it/s]


',-.;"():?!'

In [89]:
# words book
from turtle import towards


special_pattern = r'[,-\.;"\(\):\?!]'
words_book = {}
words_chapter = {}
for chapter, content in tqdm(book.items()):
    # refresh chapter
    words_chapter = {}
    # refine special character
    refine_content = re.sub(special_pattern, ".", content)
    # refine space and dot
    refine_content = re.sub(r"\s+", " ", refine_content)
    refine_content = re.sub(r"\s?\.\s?", ".", refine_content)
    refine_content = re.sub(r"[\.]+", ".", refine_content)
    refine_content = refine_content.strip()
    sentences = refine_content.split(".")
    for sentence in sentences:
        if sentence == "":
            continue
        words = sentence.split()
        counter = 0
        while counter < len(words):
            if counter + 1 < len(words):
                two_gram = " ".join(words[counter:counter + 2])
                if two_gram in dictionary:
                    counter += 2
                    if two_gram not in words_chapter:
                        words_chapter[two_gram] = 0
                    words_chapter[two_gram] += 1
                    continue
            one_gram = words[counter]
            counter += 1
            if one_gram not in words_chapter:
                words_chapter[one_gram] = 0
            words_chapter[one_gram] += 1
    # sort words chapter
    sorted_words_chapter = dict(sorted(words_chapter.items(), key=lambda item: item[1], reverse=True))
    # add chapter to book
    refine_chapter = chapter.strip().lower()
    words_book[refine_chapter] = sorted_words_chapter
    

100%|██████████| 33/33 [00:00<00:00, 253.55it/s]


In [102]:
# write to file
word_folder = "words"
if not os.path.exists(os.path.join(folder, word_folder)):
    os.mkdir(os.path.join(folder, word_folder))
for chapter_name, words_chapter in words_book.items():
    with open(os.path.join(folder, word_folder, chapter_name), "w") as file:
        for word, word_count in words_chapter.items():
            line = f"{word_count}\t-\t{word}: \n"
            file.write(line)